## Camilo' Fuentes Assignment - Toronto's Neighborhood Post Codes Classification by Clusterisation using K-Mens method

#### First of all we nee dto retrieve the information about Toront's neighborhoods from wikipedia...

_Use this in case you need to instal lxml:_

In [1]:
#!pip install lxml

Now we import pandas...

In [2]:
import pandas as pd
import numpy as np

Then we set the url and extract the first table from it using pandas and verify that the info matches...

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(url ,header=0)[0]
tables.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Then we remove the rows that haven't been assigned...

In [4]:
toronto = tables[tables['Borough'] != 'Not assigned']
toronto.sort_index(inplace=True)
toronto

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


And finally we group by postcode and borough to concatenate the neighborhoods' names.

In [5]:
merge = toronto.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
#merge[merge['Postcode'] == 'M4B']
merge

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


### An now we show the shape of the merge.

In [6]:
print(merge.shape, len(merge['Postcode'].unique()))

(103, 3) 103


## Now let's get the coordinates..

_Use this if you need to instal geocoder..._

In [7]:
#!pip install geocoder
#!pip install geopy

In [8]:
import numpy as np
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values, thanks Shilpik Balpande...

## I'm gonna try to create an heursitic method to catch the geolocation values since the server dropped my connection when I tried to force it to retireve a value for every query... 

### Initialization: we're gonna create a dataframe of geolocation with zeros where no data has been retrieved...

In [9]:
coordinates = pd.DataFrame(columns=['Postcode', 'latitude', 'longitude'])
extra = 'Toronto'

for pc in merge['Postcode']:

    postal_code = str(pc)
    borough = 'merge[merge['Postcode'] == pc]['Borough'].to_string(index=False)'
    location = None
    
    address = '{}, {}, {}'.format(postal_code, borough ,extra)
    #print(address)
    
    try:
        geolocator = Nominatim(user_agent="Camilo-Notebook")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    except:
        latitude = 0
        longitude = 0
    
    temp = pd.DataFrame({"Postcode": [pc] , 
                    "latitude": [latitude],
                    "longitude": [longitude]})
    
    coordinates = coordinates.append(temp, ignore_index = True)

Unfound = coordinates[coordinates['latitude'] == 0]['Postcode'].count()
print('The number of unfound coordinates is: ', Unfound)

The number of unfound coordinates is:  103


### Now let's fill the voids in our coordinates...

In [10]:
max_iterations = 10

while (0 in coordinates.values):
    print(max_iterations, ' iterations left...')
    Unfound = coordinates[coordinates['latitude'] == 0]['Postcode'].count()
    print('The new number of unfound coordinates is: ', Unfound)
    for pc in coordinates[coordinates['latitude'] == 0]['Postcode']:
        
        postal_code = str(pc)
        borough = 'Toronto'
        address = '{}, {}, {}'.format(postal_code, borough, extra)

        try:
            geolocator = Nominatim(user_agent="Camilo-Notebook")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        except:
            latitude = 0
            longitude = 0

        coordinates.loc[coordinates['Postcode'] == pc, ['latitude','longitude']] = [latitude, longitude]
    
    max_iterations = max_iterations - 1    
    if max_iterations <= 0:
        break

10  iterations left...
The new number of unfound coordinates is:  87
9  iterations left...
The new number of unfound coordinates is:  85
8  iterations left...
The new number of unfound coordinates is:  85
7  iterations left...
The new number of unfound coordinates is:  85
6  iterations left...
The new number of unfound coordinates is:  85
5  iterations left...
The new number of unfound coordinates is:  85
4  iterations left...
The new number of unfound coordinates is:  85
3  iterations left...
The new number of unfound coordinates is:  85
2  iterations left...
The new number of unfound coordinates is:  85
1  iterations left...
The new number of unfound coordinates is:  85


## Since this didn't work, i'll just use the csv file that already contains the geolocation...

_(It seems that for some reason there is a problem with the call...)_

In [9]:
locations = pd.read_csv("http://cocl.us/Geospatial_data")
locations

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## Renaming columns in samples and merging...

In [10]:
merge.rename(columns={'Postcode':'PostalCode'}, inplace=True)
locations.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df = pd.merge(merge, locations, on='PostalCode')
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


Let's compare with the example...

In [11]:
df[df['PostalCode'].isin(['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A'])]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
38,M4G,East York,Leaside,43.709060,-79.363452
43,M4M,East Toronto,Studio District,43.659526,-79.340923
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


After checking at wikipedia we can see that some differences (such as seen for M5A) are due to changes at the page...

## Now it's time to implement the clustering method, since the only relevant data we have here is latitude and longitude we will use distance to the centroids as our parameter...

In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [13]:

# create map of Toronto using latitude and longitude values
latitude = 43.72
longitude = -79.35

f = folium.Figure(width=500, height=300)

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10).add_to(f)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Let's first create an overall view of what are we about to cluster to try to pciture an idea of how it's gonna end...

Now we instal the packages taht we weill need...

In [14]:
#pip install webcolors==1.3

In [15]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import random
import webcolors

*We need to normaliza the data...*

In [16]:
X = df[['Latitude', 'Longitude']]
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet[0:5]

array([[1.95523876, 2.09777597],
       [1.53094905, 2.44798852],
       [1.12942801, 2.15613628],
       [1.271543  , 1.86437197],
       [1.3126078 , 1.6310228 ]])

We create a function that assign random colors for later use...

In [89]:
def random_color():
    R = random.randint(0,250)
    G = random.randint(0,250)
    B = random.randint(0,250)
    return webcolors.rgb_to_hex((R, G, B))

After some itertations, I chose to use 7 clusters since it mad emor sense (own criteria) and 50 random start points to make sure the algorithm has more consistency.

In [103]:
clusters = 7
k_means = KMeans(init = "k-means++", n_clusters = clusters, n_init = 50)

We fit the model and test the colors for the labels...

In [104]:
k_means.fit(X)
labels = k_means.labels_
#print(labels)
df['labels'] = labels
colors = []
for x in range(clusters): colors.append(random_color())
colors

['#1dd626', '#c0d5cb', '#8bf66d', '#5fc4f2', '#c57d26', '#3d2cc0', '#37ae67']

We plot again but now assigning every post code with his own label...

In [105]:
# create map of Toronto using latitude and longitude values
latitude = 43.72
longitude = -79.35

g = folium.Figure(width=1100, height=700)

map_cluster = folium.Map(location=[latitude, longitude], zoom_start=10.5).add_to(g)

# add markers to map
for lat, lng, neigh, lab in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['labels']):
    neigh = folium.Popup(neigh, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=neigh,
        color='black',
        fill=True,
        fill_color= colors[lab],
        fill_opacity=1,
        parse_html=False).add_to(map_cluster) 

map_cluster

## And that's it! Hope you enjoyed my notebook.

Best regards,
Camilo Fuentes.